# Trabalho TP4 Ex 1
## Grupo 27
### LCC 2024/2025
#### Gonçalo Gonçalves Barroso A102931
#### Rafaela Antunes Pereira A102527
#### Ricardo Eusebio Cerqueira A102878

In [1]:
from z3 import *

# Define os possíves estados do sistema
# START: Estado inicial
# FREE: Sem força de travagem
# STOPPING: A travar
# STOPPED: Travado
# BLOCKED: Rodas bloqueadas mas veiculo em movimento

Mode, (START, STOPPING, STOPPED, BLOCKED, FREE) = EnumSort('Mode', ('START', 'STOPPING', 'STOPPED', 'BLOCKED', 'FREE'))



### Problema 1

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    

A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  desloca-se; no modo `Stopped` o veículo está imobilizado.

A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`  do veículo em relação ao solo e a `velocidade linear das rodas` também em relação ao solo.  Assume-se que o sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades.  A dinâmica contínua está descrita  abaixo no bloco Equaçoes de Fluxo.


    a. Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos. 
        Os “switchs” (“jumps”) são uma  componente de projeto deste trabalho; cabe ao aluno definir quais devem ser estas  condições de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
    b. Modele em lógica temporal linear LT  propriedades que caracterizam o comportamento desejável do sistema. Nomeadamente 
        1. ”o veículo imobiliza-se completamente em menos de $$t$$ segundos” 
        2. “a velocidade $$V$$ diminui sempre com o tempo”.
    c. Construa o FOTS que que descreve a discretização do  modelo  que definiu em a. e codifique em SMT’s
    d. Codifique a verificação das propriedades temporais que definiu em b.

### Equações de Fluxo 

1. Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas. 
2. No contacto rodas/solo o atrito é constante porque a força de compressão é o peso; tem-se $f = a\cdot P$ sendo  $a$ a constante de atrito e $P$ o peso. Ambos são fixos e independentes do modo.
3. No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à diferença de velocidades  $F =  c\cdot (V-v)$.  A  constante de proporcionalidade $c$ depende do modo: é elevada no modo `Stopping` e baixa nos outros.
4. Existe um atrito no contacto corpo/ar  que é aproximado por uma constante positiva $\,b$.


5. As equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`,
$$
\begin{array}{rcl}
\dot{V} & = & - c\cdot(V-v) - b\\
\dot{v} & = & -a\cdot P  + c\cdot(V-v) 
\end{array}
$$   

e , no modo `Blocked`,  a dinâmica do sistema é  regida por

$$(V = v) \;\land\; (\,\dot{V}\,=\, -a\cdot P - b\,)$$


6. Tanto no modo `Blocked`  como no modo `Free`  existe um “timer” que impede que se permaneça nesses modo mais do que $\tau$ segundos. Os $\mathsf{jumps}(V,v,t,V',v',t')\,$ com origem nesses modos devem forçar esta condição.
7. No instante inicial assume-se $\,V = v\,=\,V_0$  ,  em que a velocidade $V_0$ é o “input” do problema.

In [2]:
def declare(i):
    state = {}
    state['m'] = Const('m' + str(i), Mode)
    state['vr'] = Real('vr' + str(i)) # Velocidade das rodas
    state['vc'] = Real('vc' + str(i)) # Velocidade atual
    state['t'] = Real('t' + str(i)) # Tempo
    state['timer'] = Real('timer' + str(i)) # Timer para o estado de transição
    return state 

In [3]:
# Constantes do sistema
a = 0.001        # Constante de aceleração
c = 0.1          # Constante de controle em modo normal
c_stopping = 0.95 # Constante de controle durante travagem
P = 1000         # Potência do sistema
tau = 0.1        # Limiar de tempo para transições
intervalo = 0.1  # Intervalo de tempo 

def init(s, v_init):
    return And(
        s['t'] == 0,
        s['m'] == START,
        s['vr'] == v_init,
        s['vc'] == v_init,
        s['timer'] == 0 
    )

In [4]:
def trans(s, p):
    
    # Inicialização
    st_free = And(
        s['t'] == p['t'], 
        s['vr'] == p['vr'],
        s['vc'] == p['vc'], 
        s['m'] == START, 
        p['m'] == FREE,
        s['timer'] == 0,
        p['timer'] == 0
    )
    
    # Comportamento em modo FREE (sem travagem)
    fr_free = And(
        s['m'] == FREE,
        p['m'] == FREE,
        p['vc'] == s['vc'] + (-c * (s['vc'] - s['vr'])),
        p['vr'] == s['vr'] + (-a*P + c*(s['vc'] - s['vr'])),
        p['t'] == s['t'] + intervalo,
        p['timer'] == s['timer'] + intervalo,
        p['timer'] <= tau,
        s['vr'] > 0.1
    )
    
    # Transição de FREE para STOPPING (inicio da travagem)
    fr_stopping = And( 
        s['t'] == p['t'], 
        s['vr'] == p['vr'], 
        s['vc'] == p['vc'],
        s['m'] == FREE, 
        p['m'] == STOPPING,  
        p['timer']==0,
        s['vr'] > 0.1,
        Or(
            s['timer']>=tau,
            s['vc'] - s['vr']>=0.1
        )
    )

    # Transição de STOPPING para BLOCKED (bloqueio das rodas)
    stopping_blocked = And(
        s['t'] == p['t'], 
        s['vr'] == p['vr'], 
        s['vc'] == p['vc'], 
        s['m'] == STOPPING, 
        p['m'] == BLOCKED, 
        s['vc'] < s['vr'] + 0.2,
        s['timer'] == 0,
        p['timer'] == 0,
        s['vr'] > 0.1
    )
    
    # Comportamento do carro a travar (veiculo a travar)
    stopping_stopping = And(
        s['m'] == STOPPING,
        p['m'] == STOPPING, 
        p['vc'] == s['vc'] + (-c_stopping * (s['vc'] - s['vr'])), 
        p['vr'] == s['vr'] + (-a * P + c_stopping * (s['vc'] - s['vr'])),
        s['vc'] > s['vr'] + 0.2,
        p['t'] == s['t']+intervalo,
        s['timer'] == p['timer'],
    )
    
    # Transição de STOPPING para STOPPED (veiculo travado)
    stopping_stopped = And( 
        s['t']  == p['t'],  
        s['m']  == STOPPING, 
        p['m']  == STOPPED, 
        s['vc'] <= 0.11,
        s['vr'] <= 0.11,
        p['vc'] == 0,
        p['vr'] == 0,
        p['timer'] == 0
    )
    
    # Transição de BLOCKED para FREE (desbloqueio das rodas)
    blocked_free = And( 
        s['t'] == p['t'], 
        s['vr'] == p['vr'], 
        s['vc'] == p['vc'], 
        s['m'] == BLOCKED, 
        p['m'] == FREE, 
        s['timer'] >= tau, 
        p['timer'] == 0,
        s['vr'] > 0.1
    )
    
    # Transição de BLOCKED para BLOCKED (veiculo bloqueado)
    blocked_blocked = And(
        s['m'] == BLOCKED,
        p['m'] == BLOCKED, 
        p['vc'] == s['vr'], 
        p['vr'] == s['vr'] + (-a*P), 
        p['t'] == s['t']+intervalo,
        p['timer']==s['timer']+intervalo,
        p['timer']<=tau,
    )
    
    # Transição de BLOCKED para STOPPED (veiculo travado)
    blocked_stopped = And(  
        s['t'] == p['t'],  
        s['m'] == BLOCKED, 
        p['m'] == STOPPED,
        s['vc'] <= 0.11,
        s['vr'] <= 0.11,
        p['vc'] == 0,
        p['vr'] == 0,
        p['timer'] == 0
    )
    
    # Transição de STOPPED para STOPPED (veiculo travado)
    stopped_stopped = And(
        s['m'] == STOPPED, 
        p['m'] == STOPPED, 
        p['t'] == s['t']+intervalo,
        s['timer'] == 0,
        p['timer'] == 0,
        s['vr'] == p['vr'],
        s['vc'] == p['vc']
    )
    
    return Or(
        st_free,
        fr_free,
        fr_stopping,
        stopping_blocked,
        stopping_stopped,
        stopping_stopping,
        blocked_free,
        blocked_blocked,
        blocked_stopped,
        stopped_stopped,
        
    )


In [5]:
def fracparafloat(f):
    return float(f.numerator_as_long())/float(f.denominator_as_long())

def gera_traco(declare, init, trans, k):
    s = Solver()
    state = [declare(i) for i in range(k)]
    s.add(init(state[0], 20))
    
    for i in range(k-1):
        s.add(trans(state[i], state[i+1]))
    
    if s.check() == sat:
        m = s.model()
        
        for i in range(k):
            print("State:", i)
            for j in state[i]:
                res = m[state[i][j]]
                if res.sort() != RealSort():
                    print(j,'=', res)
                else:
                    print(j,'=', fracparafloat(res))
            print()
    else:
        print("UNSAT")

gera_traco(declare, init, trans, 21)
        

State: 0
m = START
vr = 20.0
vc = 20.0
t = 0.0
timer = 0.0

State: 1
m = FREE
vr = 20.0
vc = 20.0
t = 0.0
timer = 0.0

State: 2
m = FREE
vr = 19.0
vc = 20.0
t = 0.1
timer = 0.1

State: 3
m = STOPPING
vr = 19.0
vc = 20.0
t = 0.1
timer = 0.0

State: 4
m = STOPPING
vr = 18.95
vc = 19.05
t = 0.2
timer = 0.0

State: 5
m = BLOCKED
vr = 18.95
vc = 19.05
t = 0.2
timer = 0.0

State: 6
m = BLOCKED
vr = 17.95
vc = 18.95
t = 0.3
timer = 0.1

State: 7
m = FREE
vr = 17.95
vc = 18.95
t = 0.3
timer = 0.0

State: 8
m = FREE
vr = 17.05
vc = 18.85
t = 0.4
timer = 0.1

State: 9
m = STOPPING
vr = 17.05
vc = 18.85
t = 0.4
timer = 0.0

State: 10
m = STOPPING
vr = 17.76
vc = 17.14
t = 0.5
timer = 0.0

State: 11
m = BLOCKED
vr = 17.76
vc = 17.14
t = 0.5
timer = 0.0

State: 12
m = BLOCKED
vr = 16.76
vc = 17.76
t = 0.6
timer = 0.1

State: 13
m = FREE
vr = 16.76
vc = 17.76
t = 0.6
timer = 0.0

State: 14
m = FREE
vr = 15.86
vc = 17.66
t = 0.7
timer = 0.1

State: 15
m = STOPPING
vr = 15.86
vc = 17.66
t = 0.7
timer 

#### Prova de propriedades
- 1 - O veículo imobliza-se em menos de t segundos
- 2 - A velocidade V diminui sempre com o tempo

In [6]:
def imob_in_t_sec(state):
    '''
    Verifica se o veiculo está imobilizado em menos de t segundos, com t = 3.5
    '''
    return Implies(state['t']>=3.5,state['m']==STOPPED)

def sempreMenor(atual,prox):
    '''
    Verifica se a velocidade do veiculo diminui com o tempo
    '''
    return Implies(atual['t']<prox['t'],atual['vc']>prox['vc'])

In [7]:
def testaImob(declare,init,trans,inv,K):
    '''
    Verifica a propriedade de imobilização para K passos
    Testa se o sistema sempre para após 3.5 segundos
    '''
    for k in range(1,K+1):
        s = Solver()
    
        trace = [declare(i) for i in range(k)]
        
        s.add(init(trace[0],20))
        for i in range(k-1):
            s.add(trans(trace[i],trace[i+1]))
        
        s.add(Not(inv(trace[k-1])))
        
        if s.check() == sat:
            print("Not valid")
            return

    print ("Property MAY be valid")
    
testaImob(declare, init, trans, imob_in_t_sec, 20)

Property MAY be valid


In [8]:
def testaMenor(declare, init, trans, inv, K):
    '''
    Verifica a propriedade de diminuição constante da velocidade
    Testa se a velocidade sempre diminui durante a travagem

    '''
    for k in range(1,K+1):
        s = Solver()
        
        trace = [declare(i) for i in range(k)]
        
        s.add(init(trace[0],20))
        aux=[]
        for i in range(k-1):
            s.add(trans(trace[i],trace[i+1]))
            aux.append(Not(inv(trace[i],trace[i+1])))
        
        s.add(Or(aux))
        
        if s.check() == sat:
            print("Not valid")
            return

    print ("Property MAY be valid")
testaMenor(declare, init, trans, sempreMenor, 20)

Not valid
